<a href="https://colab.research.google.com/github/chanseongparkk/2022_Data_Analysis_Intern_practice/blob/main/07_13_05_%EC%B9%B4%EC%9D%B4%EC%A0%9C%EA%B3%B1%EA%B2%80%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 적합도 검정

* k개의 범주를 가지는 한개의 요인에 대해서 어떤 이론적 분포를 따르고 있는지 검정하는 방법

* H0: 실제 분포와 이론적 분포 간의 차이가 없다.
* H1: 실제 분포와 이론적 분포 간의 차이가 있다.

In [ ]:
# 패키지 임포트
import pandas as pd
import numpy as np
from scipy import stats
import math
from scipy.stats import shapiro
from scipy.stats import levene
from scipy.stats import bartlett

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# os에 따른 font 깨짐 제거를 위해 font 지정

import os

if os.name == "nt":
    font_family = "Malgun Gothic"
else: #mac os
    font_family = "AppleGothic"
    
# - 값이 깨지는 문제 해결을 위해 파라미터값 설정

sns.set(font = font_family, rc = {"axes.unicode_minus" : False})

import warnings
warnings.filterwarnings("ignore")

In [ ]:
### 적합도 검정
## 데이터 생성
# 4교대 근무, 1일 불량 수 조사, 각 조의 불량 수 차이 여부 검정

observed_, expected_ = [15, 35, 24, 26], [25, 25, 25, 25]
ct_1 = pd.DataFrame([observed_, expected_], columns = ["A", "B", "C", "D"],
                    index = ["관측빈도", "기대빈도"])

ct_1

###  카이제곱통계량 계산

observed = ct_1.loc["관측빈도"]
expected = ct_1.loc["기대빈도"]

chi_squared_test = (observed - expected).pow(2) / expected  # pow(2) 제곱

chi_squared_statistic = chi_squared_test.sum()
print("카이제곱통계량", chi_squared_statistic)

## 카이제곱 검정 수행(적합도 검정)
dof = (4-1) * (2-1)
chi_ = stats.chi2(dof)
chi_05 = chi_.ppf(0.95)
print("기각역 : ", chi_05)

# 카이제곱통계량 기반 검정

if chi_05 >= chi_squared_statistic :
  print("귀무가설 채택, 4개조의 불량수는 차이가 있다")
else : 
  print("귀무가설 기각, 4개조의 불량수는 차이가 없다")

p_val = 1 - chi_.cdf(chi_squared_statistic)

print('p_value:', p_val)

# 유의수준 기반 검정

if p_val >= 0.05 :
  print("귀무가설 채택, 4개조의 불량수는 차이가 있다")
else : 
  print("귀무가설 기각, 4개조의 불량수는 차이가 없다")

### chisquare : 모집단의 분포를 알고 있고, 이에 대해 검증할 때 활용 package
stats.chisquare(ct_1.loc["관측빈도"].values, f_exp = ct_1.loc["기대빈도"].values)

카이제곱통계량 8.079999999999998
기각역 :  7.814727903251179
귀무가설 기각, 4개조의 불량수는 차이가 없다
p_value: 0.04438695936738335
귀무가설 기각, 4개조의 불량수는 차이가 없다


Power_divergenceResult(statistic=8.079999999999998, pvalue=0.044386959367383315)

In [ ]:
# 교차분석하기 위한 범주형 데이터 정의

cross_data = pd.DataFrame(
                {'영양제':['복용','복용','복용','복용','복용','복용','복용','복용',
                           '복용','복용','복용','복용','복용','복용','복용','복용',
                            '복용','복용','복용','복용',
                            '미복용','미복용','미복용','미복용','미복용','미복용',
                            '미복용','미복용','미복용','미복용','미복용','미복용',
                            '미복용','미복용','미복용','미복용','미복용','미복용',
                            '미복용', '미복용'],
                  '감기여부':['유','유','유','유','유','무','무','무','무','무',
                              '무','무','무','무','무','무','무','무','무','무',
                              '유','유','유','유','유','유','유','유','유','유',
                              '유','유','무','무','무','무','무','무','무','무']
                 }
)

cross_data.head()

,영양제,감기여부
0,복용,유
1,복용,유
2,복용,유
3,복용,유
4,복용,유


In [ ]:
# 교차분석표 생성
## summary 행렬 포함한 교차표

display(pd.crosstab(cross_data["영양제"], cross_data["감기여부"], margins = True))

## 계산하기 위한 교차표

data_crosstab = pd.crosstab(cross_data["영양제"], cross_data["감기여부"], margins = False)
data_crosstab

감기여부,무,유,All
영양제,,,
미복용,8,12,20
복용,15,5,20
All,23,17,40


감기여부,무,유
영양제,,
미복용,8,12
복용,15,5


In [ ]:
# 교차표 분석
from scipy.stats import chi2_contingency

# correction = True를 적용하면 Yates' correction이 적용되어 검정통계량이 보수적으로 더 낮게 나옴

result = chi2_contingency(observed = data_crosstab, correction = False)
print("1. 카이제곱통계량 : ", result[0])
print("2. p_value : ", result[1])
print("3. df(자유도) : ", result[2])
print("4.  기대값 행렬 ")
pd.DataFrame(result[3]).rename(index = {0: "복용", 1: "미복용"}, columns = {0: "유", 1: "무"})

if 0.05 >= result[1] : #p-value
   print("귀무가설 기각, 감기약 복용과 감기 유무는 차이가 있다.")
else: 
   print("귀무가설 채택, 감기약 복용과 감기 유무는 차이가 없다.")

1. 카이제곱통계량 :  5.012787723785166
2. p_value :  0.025160759200408785
3. df(자유도) :  1
4.  기대값 행렬 
귀무가설 기각, 감기약 복용과 감기 유무는 차이가 있다.
